In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.utils.class_weight import compute_class_weight


2025-05-05 21:12:15.192154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746479535.424998      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746479535.494212      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/clear-other/cleared_one_other.csv')
# df = pd.read_csv('../data/processed/cleared_one_class.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9881 entries, 0 to 9880
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9881 non-null   int64 
 1   text        9881 non-null   object
 2   class       9881 non-null   object
dtypes: int64(1), object(2)
memory usage: 231.7+ KB


In [4]:
df.head()

,Unnamed: 0,text,class
0,1,кнопка start,соцсети
1,2,продолжение правильно моем сообществе вк ссылк...,соцсети
2,3,временем авторская телега уверенно тройке силь...,соцсети
3,4,двоюродная сестра сын антон двоюродный племянн...,личная жизнь
4,5,временем авторская телега уверенно тройке силь...,соцсети


In [5]:
# Инициализируем энкодер
encoder = LabelEncoder()

labels =encoder.fit_transform(df['class'])
df['labels'] = labels
df.head()

,Unnamed: 0,text,class,labels
0,1,кнопка start,соцсети,4
1,2,продолжение правильно моем сообществе вк ссылк...,соцсети,4
2,3,временем авторская телега уверенно тройке силь...,соцсети,4
3,4,двоюродная сестра сын антон двоюродный племянн...,личная жизнь,1
4,5,временем авторская телега уверенно тройке силь...,соцсети,4


In [6]:
class_mapping = {label: idx for idx, label in enumerate(encoder.classes_)}
class_mapping

{'другое': 0,
 'личная жизнь': 1,
 'политика': 2,
 'реклама': 3,
 'соцсети': 4,
 'спорт': 5,
 'юмор': 6}

In [7]:
df=df.drop(columns=['class'])
train_df, val_df = train_test_split(df, stratify=df['labels'], random_state=42)

In [8]:
# Находим количество примеров в каждом классе
class_counts = train_df['labels'].value_counts()
max_count = class_counts.max()

# Функция для оверсэмплинга каждого класса
resampled_dfs = []
for class_label in class_counts.index:
    class_df = train_df[train_df['labels'] == class_label]
    if len(class_df) < max_count:
        # Дублируем примеры, пока не достигнем max_count
        class_df = class_df.sample(max_count, replace=True, random_state=42)
    resampled_dfs.append(class_df)

# Объединяем обратно
train_df = pd.concat(resampled_dfs, axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

In [9]:
texts = train_df['text'].to_numpy()
labels = train_df['labels'].to_numpy()
print(Counter(labels))

Counter({5: 1974, 6: 1974, 3: 1974, 1: 1974, 2: 1974, 0: 1974, 4: 1974})


In [10]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train' : train_dataset,
    'val': val_dataset
})

In [11]:
# Токенизация
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Map:   0%|          | 0/13818 [00:00<?, ? examples/s]

Map:   0%|          | 0/2471 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets.set_format('torch')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13818
    })
    val: Dataset({
        features: ['Unnamed: 0', 'text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2471
    })
})

In [13]:
# 5. Инициализация модели (остается без изменений)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased',
    num_labels=len(class_mapping)
).to(device)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Создание модели
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", 
    num_labels=len(class_mapping)
    )


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Для multi-class
    
    # Для multi-label (раскомментировать):
    # predictions = (predictions > 0).astype(int)  # Логиты уже через sigmoid
    
    return {
        'f1 macro': f1_score(labels, predictions, average='macro'),
        'f1 micro': f1_score(labels, predictions, average='micro'),
        'f1 weighted': f1_score(labels, predictions, average='weighted'),
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions, average='macro'),
        'recall': recall_score(labels, predictions, average='macro'),
        'Loss': hamming_loss(labels, predictions)
    }

training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    eval_strategy="steps",
    log_level='info',
    do_train=True,  # Ключевое изменение!
    do_eval=True,
    logging_steps=200,
    save_steps=1500,
    eval_steps=200,
    fp16=True,  # Если GPU поддерживает
    report_to=["tensorboard"],
    metric_for_best_model='eval_f1 micro'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using auto half precision backend


In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 13,818
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Training with DataParallel so batch size has been adjusted to: 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8,640
  Number of trainable parameters = 177,858,823
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,F1 macro,F1 micro,F1 weighted,Accuracy,Precision,Recall
200,1.201800,0.333064,0.651186,0.666936,0.664151,0.666936,0.669438,0.671612
400,0.655500,0.228248,0.761865,0.771752,0.773492,0.771752,0.768927,0.769651
600,0.318300,0.196277,0.796394,0.803723,0.802657,0.803723,0.801592,0.792316
800,0.255000,0.160259,0.831045,0.839741,0.838620,0.839741,0.830467,0.832642
1000,0.204800,0.158236,0.830628,0.841764,0.841947,0.841764,0.830056,0.837137
1200,0.164100,0.147309,0.844921,0.852691,0.853849,0.852691,0.843017,0.848384
1400,0.158800,0.135573,0.853654,0.864427,0.864085,0.864427,0.861067,0.849957
1600,0.196800,0.145285,0.844741,0.854715,0.855162,0.854715,0.851637,0.840777
1800,0.243400,0.148523,0.842762,0.851477,0.852381,0.851477,0.852269,0.839069
2000,0.197300,0.143262,0.845377,0.856738,0.855882,0.856738,0.842157,0.850552


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Unnamed: 0. If text, __index_level_0__, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2471
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Unnamed: 0. If text, __index_level_0__, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2471
  Batch s

TrainOutput(global_step=8640, training_loss=0.18070496384192397, metrics={'train_runtime': 17164.4423, 'train_samples_per_second': 16.101, 'train_steps_per_second': 0.503, 'total_flos': 7.2716635579392e+16, 'train_loss': 0.18070496384192397, 'epoch': 20.0})

In [16]:
model.save_pretrained("/kaggle/working/my_bert_one_aug_classifier")
tokenizer.save_pretrained("/kaggle/working/my_bert_one_aug_classifier")

Configuration saved in /kaggle/working/my_bert_one_aug_classifier/config.json
Model weights saved in /kaggle/working/my_bert_one_aug_classifier/model.safetensors
tokenizer config file saved in /kaggle/working/my_bert_one_aug_classifier/tokenizer_config.json
Special tokens file saved in /kaggle/working/my_bert_one_aug_classifier/special_tokens_map.json


('/kaggle/working/my_bert_one_aug_classifier/tokenizer_config.json',
 '/kaggle/working/my_bert_one_aug_classifier/special_tokens_map.json',
 '/kaggle/working/my_bert_one_aug_classifier/vocab.txt',
 '/kaggle/working/my_bert_one_aug_classifier/added_tokens.json')

In [17]:
print(trainer.evaluate()) 

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Unnamed: 0. If text, __index_level_0__, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2471
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 1.9790066480636597, 'eval_f1 macro': 0.8715654241835138, 'eval_f1 micro': 0.8806151355726427, 'eval_f1 weighted': 0.880219668741596, 'eval_accuracy': 0.8806151355726427, 'eval_precision': 0.8764716077764769, 'eval_recall': 0.8678341776923343, 'eval_Loss': 0.11938486442735735, 'eval_runtime': 49.7073, 'eval_samples_per_second': 49.711, 'eval_steps_per_second': 3.118, 'epoch': 20.0}


In [18]:
new_texts = ["Анкара Месси забил гол в ворота франции", "ай Литвин красава ай чисто на кондциях залетел ай да лев", "Америка расширила список санкция против России", "купите макбук за 140к и получите наушники в подарок"]


In [1]:
def predict(text):
    # Токенизация текста
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    
    # Предсказание
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получение метки класса
    probs = torch.sigmoid(outputs.logits)
    pred_class = torch.argmax(probs).item()
    
    id2label = {
        0: 'другое',
        1: "личная жизнь",
        2: "политика", 
        3: "реклама",
        4: "соцсети",
        5: "спорт",
        6: "юмор"
    }

    for class_id, prob in enumerate(probs.cpu().numpy()[0]):
        print(f"{id2label[class_id]}: {prob:.4f}")  # 4 знака после запятой

In [2]:
class_mapping

NameError: name 'class_mapping' is not defined

In [3]:
predict('Спортивная энергия в каждом глотке — заряжайся и побеждай с новым напитком для чемпионов!')

NameError: name 'tokenizer' is not defined

In [ ]:
predict('Достигай рекордов с кроссовками, которые выбирают профи!')

In [ ]:
predict('Подпитывай чемпионский дух — белковый батончик для твоих тренировок!')